# Kernel Tuner Simulation Mode Test

This notebook is intended to examine the completeness, consistency, and advantage of simulation mode in Kernel Tuner.

In [ ]:
from pathlib import Path
import json
import re
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from warnings import warn

cachedirectory = Path("kernels/outputdata/simulation_mode")
savedirectory = Path(Path.cwd(), "figures", "simulation_mode")
mpl.rcParams["savefig.directory"] = savedirectory

In [ ]:
# arguments
kernelname = "gemm_cltune_opencl"

algorithm_displaynames = {
    'brute': "bruteforce",
    'random': "random",
}

plot_height = 6
plot_aspect = 2.0
plot_dpi = 250

In [ ]:
def get_dataframe() -> pd.DataFrame:

    # collect the relevant cachefiles
    cachefiles : list[Path] = []
    for file in cachedirectory.iterdir():
        if file.is_file() and file.name.startswith(kernelname) and file.suffix.endswith('_output.json'):
            cachefiles.append(file)

    # create a list of dictionaries of the data
    cache_dicts = list()
    for filepath in cachefiles:
        # load the data
        envfilepath = Path(str(filepath).replace('_output', '_env'))
        assert envfilepath.exists()
        try:
            cachefile = json.load(filepath.open())
            envfile = json.load(envfilepath)
        except json.JSONDecodeError:
            warn(f"Could not load cachefile {filepath.name}")
            continue
        assert isinstance(cachefile, dict)
        assert isinstance(envfile, dict)

        # combine the data into a dictionary
        device_name = cachefile['device_name']
        is_simulated = "_mode=simulated" in filepath.name
        algorithm = algorithm_displaynames[re.search(r"_alg-([a-zA-Z]+)_", filepath.name).group(1).strip()]
        # for each of the configurations, create a dictionary with the expected items
        for value in cachefile['cache'].values():
            assert isinstance(value, dict)
            cache = {}
            cache['device_name'] = device_name
            cache['backend'] = backend
            cache['backend | CUDA version'] = backend_CUDA
            cache['CUDA version | backend'] = CUDA_backend
            cache['CUDA version'] = CUDA_version
            for key in expected_keys:
                cache[key] = value[key]
            cache['total_time'] = value['compile_time'] + value['benchmark_time'] + value['framework_time']
            if observers:
                for key in observer_keys:
                    if key in value:
                        cache[key] = value[key]
            cache_dicts.append(cache)

    # create a dataframe out of the dictionaries
    df_ = pd.DataFrame(cache_dicts)
    return df_.sort_values(['CUDA version', 'backend'])